In [1]:
import time 
start=time.time()

In [2]:
#importing libraries

import gensim
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import re
import pandas as pd
from pandas import DataFrame
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#Loading pre=trained word2vec model

from gensim.models.keyedvectors import KeyedVectors

# You need to dowload google pre-trained model using below link
# https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
#Change the path according to your directory

model_path = 'D:\GoogleNews_vectors_negative300\GoogleNews_vectors_negative300.bin'   
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [4]:
# Importing the sick dataset

sick_dataset = pd.read_csv('SICK_train.txt', delimiter = "\t")
sick_dataset.loc[0:20,:]


,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.500,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.200,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.700,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.400,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.700,NEUTRAL
5,12,Two dogs are fighting,Two dogs are wrestling and hugging,4.000,NEUTRAL
6,14,A brown dog is attacking another animal in fro...,Two dogs are fighting,3.500,NEUTRAL
7,18,A brown dog is attacking another animal in fro...,Two dogs are wrestling and hugging,3.200,NEUTRAL
8,25,Nobody is riding the bicycle on one wheel,A person in a black jacket is doing tricks on ...,2.800,NEUTRAL
9,26,A person is riding the bicycle on one wheel,A man in a black jacket is doing tricks on a m...,3.700,NEUTRAL


In [5]:
# Importing first 20 lines from SICK dataset
#Calling this as source rules

source_rules=sick_dataset.loc[1:20,'sentence_A']

#adding the selected rules into a csv file
source_rules.to_csv('source_rules.csv', index=False)

In [6]:
# Importing first 20 lines from SICK dataset
#Calling this as target rules

target_rules=sick_dataset.loc[1:20,'sentence_B']

#adding the selected rules into a csv file
target_rules.to_csv('target_rules.csv', index=False)

In [7]:
# Importing Source Rules from CSV file

with open ('source_rules.csv') as file_object:
    for OCDline in file_object:
        print(OCDline)

sentence_A

A group of children is playing in the house and there is no man standing in the background

The young boys are playing outdoors and the man is smiling nearby

The kids are playing outdoors near a man with a smile

The young boys are playing outdoors and the man is smiling nearby

Two dogs are fighting

A brown dog is attacking another animal in front of the man in pants

A brown dog is attacking another animal in front of the man in pants

Nobody is riding the bicycle on one wheel

A person is riding the bicycle on one wheel

A person on a black motorbike is doing tricks with a jacket

A man with a jersey is dunking the ball at a basketball game

A man with a jersey is dunking the ball at a basketball game

The player is dunking the basketball into the net and a crowd is in background

Two people are kickboxing and spectators are not watching

Two young women are sparring in a kickboxing fight

Two young women are not sparring in a kickboxing fight

Two people are kickboxin

In [8]:
#Sentences tokenization of Source Rules

source_rules = []
with open ('source_rules.csv') as file_object:
    for trainline in file_object:
        tokens_train = sent_tokenize(trainline)
        source_rules.extend(tokens_train)

In [9]:
# Importing Target Rules 

with open ('target_rules.csv') as file_object:
    for EAline in file_object:
        print(EAline)

sentence_B

A group of kids is playing in a yard and an old man is standing in the background

The kids are playing outdoors near a man with a smile

A group of kids is playing in a yard and an old man is standing in the background

A group of kids is playing in a yard and an old man is standing in the background

Two dogs are wrestling and hugging

Two dogs are fighting

Two dogs are wrestling and hugging

A person in a black jacket is doing tricks on a motorbike

A man in a black jacket is doing tricks on a motorbike

A person is riding the bicycle on one wheel

The ball is being dunked by a man with a jersey at a basketball game

A man who is playing dunks the basketball into the net and a crowd is in background

A man with a jersey is dunking the ball at a basketball game

Two people are kickboxing and spectators are watching

Two women are sparring in a kickboxing match

Two women are sparring in a kickboxing match

Two young women are not sparring in a kickboxing fight

Two peopl

In [10]:
#Sentences tokenization of Target Rules

target_rules = []
with open ('target_rules.csv') as file_object:
    for trainline in file_object:
        tokens_train = sent_tokenize(trainline)
        target_rules.extend(tokens_train)  

In [11]:
#Setting Parameters for model

class DocSim(object):
    def __init__(self, w2v_model , stopwords=[]):
        self.w2v_model = w2v_model
        self.stopwords = stopwords

    def vectorize(self, doc):
        """Identify the vector values for each word in the given document"""
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors

        vector = np.mean(word_vecs, axis=0)
        return vector

    def jaccard_sim(self,vecA, vecB):
            """Find the jaccard similarity distance between two vectors."""
            num=np.dot(vecA,vecB)
            denum1=((np.linalg.norm(vecA))**2)+((np.linalg.norm(vecB))**2)
            denum2=np.dot(vecA,vecB)
            jsim=num/(denum1-denum2)
            if np.isnan(np.sum(jsim)):                   #Test element-wise for NaN and return result as a boolean array.
                return 0
            return jsim 
    def calculate_similarity(self, source_rules, target_rules=[], threshold=0.8):
        """Calculates & returns similarity scores between given source rules & all
        the target rules"""
        if isinstance(target_rules, str):
            target_rules = [target_rules]


        source_vec = self.vectorize(source_rules)
        results = []
        
        for rule in target_rules:
            target_vec = self.vectorize(rule)
            sim_score = self.jaccard_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({
                    'Jaccard Sim Score':sim_score,
                    'Target Rule':rule
                })

                
            # Sort results by score in desc order
            results.sort(key=lambda k : k['Jaccard Sim Score'] , reverse=True)

        return results


In [12]:
ds = DocSim(w2v_model)

In [13]:
#Calculate the similarity score between a source rule & a target rule.


if isinstance(source_rules, str):
    source_rules = [source_rules]
   

# This will return one target rules text with a similarity score

for rule in source_rules:
    sim_scores= ds.calculate_similarity(rule, target_rules)

    
    
    # Source Rules are OCD Rules
    # Target Rules are EA Rules
    
    # Printing the output in text file
    
    print("Source rule: {} \n\nSimilarity with Target Rule is \n\n {}\n".format(rule, sim_scores) , file=open("output.txt", "a"))
    print("\n")
    
    
    # Printing output in Jupyter
    
    print("Source rule: {} \n\nSimilarity with Target Rule is \n\n {}\n".format(rule, sim_scores) )
    print("\n")

C:\Users\NR\.conda\envs\gpuversion\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\NR\.conda\envs\gpuversion\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)




Source rule: sentence_A 

Similarity with Target Rule is 

 []





Source rule: A group of children is playing in the house and there is no man standing in the background 

Similarity with Target Rule is 

 [{'Jaccard Sim Score': 0.8077252198590777, 'Target Rule': 'A group of kids is playing in a yard and an old man is standing in the background'}, {'Jaccard Sim Score': 0.8077252198590777, 'Target Rule': 'A group of kids is playing in a yard and an old man is standing in the background'}, {'Jaccard Sim Score': 0.8077252198590777, 'Target Rule': 'A group of kids is playing in a yard and an old man is standing in the background'}]





Source rule: The young boys are playing outdoors and the man is smiling nearby 

Similarity with Target Rule is 

 [{'Jaccard Sim Score': 0.8176301717698304, 'Target Rule': 'The kids are playing outdoors near a man with a smile'}]





Source rule: The kids are playing outdoors near a man with a smile 

Similarity with Target Rule is 

 [{'Jaccard Sim S

In [14]:
done=time.time()
elapsed=done-start
print(elapsed)

92.77523422241211
